In [ ]:
# Imports
from fucciphase.phase import compute_phase_trigo
from fucciphase.plot import plot_phase
from fucciphase.simulator import simulate_single_track
from fucciphase.utils import normalize_channels

## Generate data

In [ ]:
df = simulate_single_track()
print(df)

## Normalize the last two channels

In [ ]:
new_channels_name = normalize_channels(df, ["MEAN_INTENSITY_CH3", "MEAN_INTENSITY_CH4"])
print(df[new_channels_name])

## Compute the cell cycle percentage for each spot

In [ ]:
new_channels_name = compute_phase_trigo(df, "MEAN_INTENSITY_CH3", "MEAN_INTENSITY_CH4")
print(df[new_channels_name])

## Plot results for track 85

In [ ]:
# filter dataframe for TRACK_ID 85
df85 = df[df["TRACK_ID"] == 85]

# plot phase
plot_phase(df85, "MEAN_INTENSITY_CH3", "MEAN_INTENSITY_CH4")

In [ ]:
df["PHASE"]

In [ ]:
df["MEAN_INTENSITY_UNIQUE_VALUE"]